## Training and Testing

### 1. Importing Dependencies

In [1]:
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gym.wrappers import GrayScaleObservation, ResizeObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.monitor import Monitor
import os

### 2. Setting up Environment for Training

In [2]:
# 1. Create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# 2. Simplify the controls 
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# 3. Grayscale
env = GrayScaleObservation(env, keep_dim=True)
# 4. Resize
env = ResizeObservation(env, shape=84)
# 5. For Tensorboard Log
env = Monitor(env)
# 6. Wrap inside the Dummy Environment
env = DummyVecEnv([lambda: env]) 
# 7. Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

### 3. Training the Agent

In [ ]:
# Training and Logging Callback Function Class
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'saved_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [5]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [ ]:
# Create agent model
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.00003, seed=4)

In [ ]:
# Start model training
model.learn(total_timesteps=100000, callback=callback)

### 4. Testing the Model

In [4]:
# Load model
model = PPO.load('./model_100000', env=env, clip_range=0.2)

C:\Users\ANMOL\miniconda3\Lib\site-packages\stable_baselines3\common\save_util.py:166: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
  warnings.warn(
C:\Users\ANMOL\miniconda3\Lib\site-packages\stable_baselines3\common\save_util.py:166: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
  warnings.warn(


Wrapping the env in a VecTransposeImage.


In [5]:
#Test Model
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action, _ = model.predict(state)
        state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

C:\Users\ANMOL\miniconda3\Lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in scalar subtract
  return (self.ram[0x86] - self.ram[0x071c]) % 256


Episode:1 Score:[1998.]
Episode:2 Score:[1771.]
Episode:3 Score:[1850.]
Episode:4 Score:[1923.]
Episode:5 Score:[1590.]
Episode:6 Score:[715.]
Episode:7 Score:[1063.]
Episode:8 Score:[713.]
Episode:9 Score:[1492.]
Episode:10 Score:[1870.]


### 5. Opening Tensorboard

In [ ]:
LOG_NAME = 'PPO_2'
TB_LOG = os.path.join(LOG_DIR, LOG_NAME)

!tensorboard --logdir={TB_LOG}